**Pytorch BERT baseline**

In this version, I convert https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic into pytorch version

**Please upvote the kernel if you find it helpful**

### Install HuggingFace transformers & sacremoses dependency

As we are not allowed to use internet I've created required datasets and commands to setup Hugging Face Transformers setup in offline mode. You can find the required github codebases in the datasets.

* sacremoses dependency - https://www.kaggle.com/axel81/sacremoses
* transformers - https://www.kaggle.com/axel81/transformers

In [1]:
!pip install ../input/sacremoses/sacremoses-master/
!pip install ../input/transformers/transformers-master/

Processing /kaggle/input/sacremoses/sacremoses-master
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=882724 sha256=e3aedc3b79378003d1fad04b7c5e9e22674249e30e07bb03c4036709e4899c10
  Stored in directory: /tmp/.cache/pip/wheels/82/48/4b/05cb49d913a40c9d76f97931cd747d72fb17a77b0f6415cdba
Successfully built sacremoses
Processing /kaggle/input/transformers/transformers-master
  Created wheel for transformers: filename=transformers-2.1.1-cp36-none-any.whl size=334890 sha256=2e90d71f4a375d6fc240964676a9cade12ecc9ad55add3c5078457b63b523a8a
  Stored in directory: /tmp/.cache/pip/wheels/ce/f3/1a/ee7248890cb4b8e8975988b1a67999e2d09ef54ce8ee815255
Successfully built transformers


### Required Imports

I've added imports that will be used in training too

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
DATA_DIR = '../input/google-quest-challenge'

In [3]:
!ls ../input

bert-base-uncased	quest-models  transformers
google-quest-challenge	sacremoses


In [4]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

qa_id  question_asker_intent_understanding  question_body_critical  \
0     39                              0.00308                 0.00308   
1     46                              0.00448                 0.00448   
2     70                              0.00673                 0.00673   
3    132                              0.01401                 0.01401   
4    200                              0.02074                 0.02074   

   question_conversational  question_expect_short_answer  \
0                  0.00308                       0.00308   
1                  0.00448                       0.00448   
2                  0.00673                       0.00673   
3                  0.01401                       0.01401   
4                  0.02074                       0.02074   

   question_fact_seeking  question_has_commonly_accepted_answer  \
0                0.00308                                0.00308   
1                0.00448                                0.00448   
2                0.00673                                0.00673   
3                0.01401                                0.01401   
4                0.02074                                0.02074   

   question_interestingness_others  question_interestingness_self  \
0                          0.00308                        0.00308   
1                          0.00448                        0.00448   
2                          0.00673                        0.00673   
3                          0.01401                        0.01401   
4                          0.02074                        0.02074   

   question_multi_intent  ...  question_well_written  answer_helpful  \
0                0.00308  ...                0.00308         0.00308   
1                0.00448  ...                0.00448         0.00448   
2                0.00673  ...                0.00673         0.00673   
3                0.01401  ...                0.01401         0.01401   
4                0.02074  ...                0.02074         0.02074   

   answer_level_of_information  answer_plausible  answer_relevance  \
0                      0.00308           0.00308           0.00308   
1                      0.00448           0.00448           0.00448   
2                      0.00673           0.00673           0.00673   
3                      0.01401           0.01401           0.01401   
4                      0.02074           0.02074           0.02074   

   answer_satisfaction  answer_type_instructions  answer_type_procedure  \
0              0.00308                   0.00308                0.00308   
1              0.00448                   0.00448                0.00448   
2              0.00673                   0.00673                0.00673   
3              0.01401                   0.01401                0.01401   
4              0.02074                   0.02074                0.02074   

   answer_type_reason_explanation  answer_well_written  
0                         0.00308              0.00308  
1                         0.00448              0.00448  
2                         0.00673              0.00673  
3                         0.01401              0.01401  
4                         0.02074              0.02074  

[5 rows x 31 columns]

In [5]:
target_columns = sub.columns.values[1:].tolist()
target_columns

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

### Define dataset

In [6]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

qa_id                                     question_title  \
0      0  What am I losing when using extension tubes in...   
1      1  What is the distinction between a city and a s...   
2      2  Maximum protusion length for through-hole comp...   
3      3              Can an affidavit be used in Beit Din?   
4      5       How do you make a binary image in Photoshop?   

                                       question_body question_user_name  \
0  After playing around with macro photography on...               ysap   
1  I am trying to understand what kinds of places...      russellpierce   
2  I'm working on a PCB that has through-hole com...          Joe Baker   
3  An affidavit, from what i understand, is basic...         Scimonster   
4  I am trying to make a binary image. I want mor...            leigero   

                                  question_user_page  \
0         https://photo.stackexchange.com/users/1024   
1           https://rpg.stackexchange.com/users/8774   
2  https://electronics.stackexchange.com/users/10157   
3       https://judaism.stackexchange.com/users/5151   
4  https://graphicdesign.stackexchange.com/users/...   

                                              answer answer_user_name  \
0  I just got extension tubes, so here's the skin...           rfusca   
1  It might be helpful to look into the definitio...     Erik Schmidt   
2  Do you even need grooves?  We make several pro...      Dwayne Reid   
3  Sending an "affidavit" it is a dispute between...    Y     e     z   
4  Check out Image Trace in Adobe Illustrator. \n...             q2ra   

                                    answer_user_page  \
0         https://photo.stackexchange.com/users/1917   
1           https://rpg.stackexchange.com/users/1871   
2  https://electronics.stackexchange.com/users/64754   
3       https://judaism.stackexchange.com/users/4794   
4  https://graphicdesign.stackexchange.com/users/...   

                                                 url   category  ...  \
0  http://photo.stackexchange.com/questions/9169/...  LIFE_ARTS  ...   
1  http://rpg.stackexchange.com/questions/47820/w...    CULTURE  ...   
2  http://electronics.stackexchange.com/questions...    SCIENCE  ...   
3  http://judaism.stackexchange.com/questions/551...    CULTURE  ...   
4  http://graphicdesign.stackexchange.com/questio...  LIFE_ARTS  ...   

  question_well_written  answer_helpful  answer_level_of_information  \
0              1.000000        1.000000                     0.666667   
1              0.888889        0.888889                     0.555556   
2              0.777778        0.777778                     0.555556   
3              0.888889        0.833333                     0.333333   
4              1.000000        1.000000                     0.666667   

   answer_plausible  answer_relevance  answer_satisfaction  \
0          1.000000          1.000000             0.800000   
1          0.888889          0.888889             0.666667   
2          1.000000          1.000000             0.666667   
3          0.833333          1.000000             0.800000   
4          1.000000          1.000000             0.800000   

   answer_type_instructions  answer_type_procedure  \
0                       1.0               0.000000   
1                       0.0               0.000000   
2                       0.0               0.333333   
3                       0.0               0.000000   
4                       1.0               0.000000   

   answer_type_reason_explanation  answer_well_written  
0                        0.000000             1.000000  
1                        0.666667             0.888889  
2                        1.000000             0.888889  
3                        1.000000             1.000000  
4                        1.000000             1.000000  

[5 rows x 41 columns]

In [7]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

qa_id                                     question_title  \
0     39  Will leaving corpses lying around upset my pri...   
1     46         Url link to feature image in the portfolio   
2     70  Is accuracy, recoil or bullet spread affected ...   
3    132     Suddenly got an I/O error from my external HDD   
4    200  Passenger Name - Flight Booking Passenger only...   

                                       question_body question_user_name  \
0  I see questions/information online about how t...              Dylan   
1  I am new to Wordpress. i have issue with Featu...                Anu   
2  To experiment I started a bot game, toggled in...             Konsta   
3  I have used my Raspberry Pi as a torrent-serve...           robbannn   
4  I have bought Delhi-London return flights for ...               Amit   

                                  question_user_page  \
0       https://gaming.stackexchange.com/users/64471   
1    https://wordpress.stackexchange.com/users/72927   
2       https://gaming.stackexchange.com/users/37545   
3  https://raspberrypi.stackexchange.com/users/17341   
4       https://travel.stackexchange.com/users/29089   

                                              answer answer_user_name  \
0  There is no consequence for leaving corpses an...        Nelson868   
1  I think it is possible with custom fields.\n\n...            Irina   
2  You do not have armour in the screenshots. Thi...   Damon Smithies   
3  Your Western Digital hard drive is disappearin...      HeatfanJohn   
4  I called two persons who work for Saudia (tick...    Nean Der Thal   

                                   answer_user_page  \
0      https://gaming.stackexchange.com/users/97324   
1   https://wordpress.stackexchange.com/users/27233   
2      https://gaming.stackexchange.com/users/70641   
3  https://raspberrypi.stackexchange.com/users/1311   
4      https://travel.stackexchange.com/users/10051   

                                                 url    category  \
0  http://gaming.stackexchange.com/questions/1979...     CULTURE   
1  http://wordpress.stackexchange.com/questions/1...  TECHNOLOGY   
2  http://gaming.stackexchange.com/questions/2154...     CULTURE   
3  http://raspberrypi.stackexchange.com/questions...  TECHNOLOGY   
4  http://travel.stackexchange.com/questions/4704...     CULTURE   

                            host  
0       gaming.stackexchange.com  
1    wordpress.stackexchange.com  
2       gaming.stackexchange.com  
3  raspberrypi.stackexchange.com  
4       travel.stackexchange.com

In [8]:
import torch
#import torch.utils.data as data
from torchvision import datasets, models, transforms
from transformers import *
from sklearn.utils import shuffle
import random
from math import floor, ceil
from sklearn.model_selection import GroupKFold

MAX_LEN = 512
#MAX_Q_LEN = 250
#MAX_A_LEN = 259
SEP_TOKEN_ID = 102

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased/')

    def __getitem__(self, index):
        row = self.df.iloc[index]
        token_ids, seg_ids = self.get_token_ids(row)
        if self.labeled:
            labels = self.get_label(row)
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

    def __len__(self):
        return len(self.df)

    def select_tokens(self, tokens, max_num):
        if len(tokens) <= max_num:
            return tokens
        if self.train_mode:
            num_remove = len(tokens) - max_num
            remove_start = random.randint(0, len(tokens)-num_remove-1)
            return tokens[:remove_start] + tokens[remove_start + num_remove:]
        else:
            return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]

    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=30, q_max_len=239, a_max_len=239):
        t = self.tokenizer.tokenize(title)
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)

        t_len = len(t)
        q_len = len(q)
        a_len = len(a)

        if (t_len+q_len+a_len+4) > max_sequence_length:

            if t_max_len > t_len:
                t_new_len = t_len
                a_max_len = a_max_len + floor((t_max_len - t_len)/2)
                q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
            else:
                t_new_len = t_max_len

            if a_max_len > a_len:
                a_new_len = a_len 
                q_new_len = q_max_len + (a_max_len - a_len)
            elif q_max_len > q_len:
                a_new_len = a_max_len + (q_max_len - q_len)
                q_new_len = q_len
            else:
                a_new_len = a_max_len
                q_new_len = q_max_len


            if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))

            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]

        return t, q, a
        
    def get_token_ids(self, row):
        t_tokens, q_tokens, a_tokens = self.trim_input(row.question_title, row.question_body, row.answer)

        tokens = ['[CLS]'] + t_tokens + ['[SEP]'] + q_tokens + ['[SEP]'] + a_tokens + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            token_ids += [0] * (MAX_LEN - len(token_ids))
        ids = torch.tensor(token_ids)
        seg_ids = self.get_seg_ids(ids)
        return ids, seg_ids
    
    def get_seg_ids(self, ids):
        seg_ids = torch.zeros_like(ids)
        seg_idx = 0
        first_sep = True
        for i, e in enumerate(ids):
            seg_ids[i] = seg_idx
            if e == SEP_TOKEN_ID:
                if first_sep:
                    first_sep = False
                else:
                    seg_idx = 1
        pad_idx = torch.nonzero(ids == 0)
        seg_ids[pad_idx] = 0

        return seg_ids

    def get_label(self, row):
        #print(row[target_columns].values)
        return torch.tensor(row[target_columns].values.astype(np.float32))

    def collate_fn(self, batch):
        token_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
    
        if self.labeled:
            labels = torch.stack([x[2] for x in batch])
            return token_ids, seg_ids, labels
        else:
            return token_ids, seg_ids

def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')
    ds_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=ds_test.collate_fn, drop_last=False)
    loader.num = len(df)
    
    return loader
        
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')
    df = shuffle(df, random_state=1234)
    #split_index = int(len(df) * (1-val_percent))
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    #print(df_val.head())
    #df_train = df[:split_index]
    #df_val = df[split_index:]

    print(df_train.shape)
    print(df_val.shape)

    ds_train = QuestDataset(df_train)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=ds_train.collate_fn, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, collate_fn=ds_val.collate_fn, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader

def test_train_loader():
    loader, _ = get_train_val_loaders(4, 4, 1)
    for ids, seg_ids, labels in loader:
        print(ids)
        print(seg_ids.numpy())
        print(labels)
        break
def test_test_loader():
    loader = get_test_loader(4)
    for ids, seg_ids in loader:
        print(ids)
        print(seg_ids)
        break

In [9]:
test_train_loader()

(4863, 41)
(1216, 41)
tensor([[  101,  9776, 11287,  ...,     0,     0,     0],
        [  101,  2129,  2079,  ...,  1998,  5454,   102],
        [  101,  2129,  2064,  ...,  2467,  2342,   102],
        [  101,  3180,  3670,  ...,     0,     0,     0]])
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 0 0 0]]
tensor([[0.3333, 0.3333, 0.0000, 1.0000, 1.0000, 1.0000, 0.6667, 0.3333, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         1.0000, 0.0000, 0.3333, 0.6667, 0.3333, 0.6667, 1.0000, 0.6000, 1.0000,
         0.0000, 1.0000, 0.6667],
        [1.0000, 0.6667, 0.0000, 0.6667, 0.6667, 0.6667, 0.7778, 0.7778, 0.6667,
         0.0000, 0.6667, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 1.0000, 0.8889, 1.0000, 1.0000, 1.0000, 0.6667,
         0.0000, 1.0000, 1.0000],
        [1.0000, 0.7778, 0.0000, 1.0000, 0.6667, 1.0000, 0.5556, 0.5556, 0.0000,
         0.0000, 0.3333, 0.3333, 

## Build Model

In [10]:
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F


class QuestModel(nn.Module):
    def __init__(self, n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
        self.bert_model = BertModel.from_pretrained('../input/bert-base-uncased/')    
        self.fc = nn.Linear(768, n_classes)

    def forward(self, ids, seg_ids):
        attention_mask = (ids > 0)
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        #print(layers[-1][0].size())
        #print(pool_out.size())

        #out = F.dropout(layers[-1][:, 0, :], p=0.2, training=self.training)
        out =  F.dropout(pool_out, p=0.2, training=self.training)
        logit = self.fc(out)
        return logit
    
def test_model():
    x = torch.tensor([[1,2,3,4,5, 0, 0], [1,2,3,4,5, 0, 0]])
    seg_ids = torch.tensor([[0,0,0,0,0, 0, 0], [0,0,0,0,0, 0, 0]])
    model = QuestModel()

    y = model(x, seg_ids)
    print(y)

In [11]:
!ls ../input

bert-base-uncased	quest-models  transformers
google-quest-challenge	sacremoses


In [12]:
def create_model(model_file):
    model = QuestModel()
    model.load_state_dict(torch.load(model_file))
    model = model.cuda()
    #model = DataParallel(model)
    return model

def create_models():
    models = []
    for i in range(5):
        model = create_model(f'../input/quest-models/best_{i}.pth')
        model.eval()
        models.append(model)
    return models

In [13]:
from tqdm import tqdm
import torch
def predict(models, test_loader):
    all_scores = []
    with torch.no_grad():
        for ids, seg_ids in tqdm(test_loader, total=test_loader.num // test_loader.batch_size):
            ids, seg_ids = ids.cuda(), seg_ids.cuda()
            scores = []
            for model in models:
                outputs = torch.sigmoid(model(ids, seg_ids)).cpu()
                scores.append(outputs)
            all_scores.append(torch.mean(torch.stack(scores), 0))

    all_scores = torch.cat(all_scores, 0).numpy()
    
    return all_scores

In [14]:
test_loader = get_test_loader(batch_size=32)

In [15]:
models = create_models()

In [16]:
preds = predict(models, test_loader)

15it [00:52,  3.51s/it]


In [17]:
preds[:1]

array([[0.9544323 , 0.69649154, 0.215785  , 0.5413474 , 0.52537286,
        0.49783736, 0.68470657, 0.6549454 , 0.6422643 , 0.0064959 ,
        0.8433417 , 0.7326375 , 0.03308813, 0.033637  , 0.01292908,
        0.00688374, 0.08041314, 0.07203815, 0.83665323, 0.00487517,
        0.92573756, 0.93559754, 0.6425709 , 0.9787086 , 0.97245646,
        0.8553497 , 0.03412033, 0.04214398, 0.9541912 , 0.94158155]],
      dtype=float32)

### Generate Submission

In [18]:
sub[target_columns] = preds

In [19]:
sub.to_csv('submission.csv', index=False)

In [20]:
sub.head()

qa_id  question_asker_intent_understanding  question_body_critical  \
0     39                             0.954432                0.696492   
1     46                             0.892354                0.507655   
2     70                             0.930704                0.712337   
3    132                             0.894929                0.485413   
4    200                             0.914849                0.411939   

   question_conversational  question_expect_short_answer  \
0                 0.215785                      0.541347   
1                 0.004250                      0.830887   
2                 0.011968                      0.826424   
3                 0.006502                      0.764054   
4                 0.041812                      0.785519   

   question_fact_seeking  question_has_commonly_accepted_answer  \
0               0.525373                               0.497837   
1               0.845054                               0.968843   
2               0.933801                               0.955383   
3               0.838100                               0.930093   
4               0.630206                               0.719074   

   question_interestingness_others  question_interestingness_self  \
0                         0.684707                       0.654945   
1                         0.549142                       0.463446   
2                         0.614796                       0.532360   
3                         0.597532                       0.445274   
4                         0.639411                       0.567919   

   question_multi_intent  ...  question_well_written  answer_helpful  \
0               0.642264  ...               0.925738        0.935598   
1               0.138053  ...               0.760521        0.949986   
2               0.294271  ...               0.895681        0.940572   
3               0.062542  ...               0.756667        0.952818   
4               0.093543  ...               0.729010        0.922851   

   answer_level_of_information  answer_plausible  answer_relevance  \
0                     0.642571          0.978709          0.972456   
1                     0.648884          0.972678          0.977684   
2                     0.651745          0.977163          0.977166   
3                     0.675969          0.978619          0.979441   
4                     0.606018          0.967153          0.962247   

   answer_satisfaction  answer_type_instructions  answer_type_procedure  \
0             0.855350                  0.034120               0.042144   
1             0.879420                  0.937851               0.094043   
2             0.867441                  0.072033               0.053226   
3             0.877940                  0.791640               0.192637   
4             0.808672                  0.173027               0.106724   

   answer_type_reason_explanation  answer_well_written  
0                        0.954191             0.941582  
1                        0.052525             0.901595  
2                        0.912686             0.934617  
3                        0.705393             0.920941  
4                        0.665800             0.920765  

[5 rows x 31 columns]